In [1]:
import pandas as pd
import numpy as np 
import datetime
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products_ext.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")

#create a reference dataframe
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})

import ast
for colname in inventory.columns[2:]:
    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]
    #converting inventory rows to list of tuples from string
inventory['last_delivery'] = inventory['day'].shift(periods = 1)

tup_list2 = [('Blauwe bessen', '150g'), ('Rundergehakt', '300g '), ('Unox Gelderse rookworst', '285g'), ('Biologisch rundergehakt', '300g')]
for pair in tup_list2:
    
    transactions.loc[transactions[(transactions['description'] == pair[0]) & (transactions['size'] == pair[1])].index, 'description'] = pair[0] + '.1'

    products.loc[products[(products['description'] == pair[0]) & (products['size'] == pair[1])].index, 'description'] = pair[0] + '.1'

In [2]:
product = 'Biologische brocolli'

In [14]:
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["day"])
    return converted_date

inventory["date"] = inventory.apply(date_converter, axis = 1) #convert day of year to actual timestamp
def week_of_year(row):
    week_nr = datetime.date(row["date"].year, row["date"].month, row["date"].day).isocalendar()[1]
    return week_nr

inventory["week"] = inventory.apply(week_of_year, axis = 1)
def check_discount(row):
    if row["week"] in promotions_filter["week"].values:
        discount_amount = promotions_filter[promotions_filter["week"] == row["week"]]["discount"]
        return discount_amount.values[0]
    else:
        return 0

def total_inv(row):
    total = 0
    if len(row[product]) > 0:
        for batch in row[product]:
            total += batch[1]
    return total
#adds total nr items in inventory for the filtered dataframe

def input_inv(row):
    for element_list in filter_inventory[product]:
        if len(element_list) > 0:
            for batch in element_list:
                if batch[0] == row["best before"]:
                    return batch[1]

transactions["day"] = pd.to_datetime(transactions["day"])
first_restock = df_date[df_date["day"] == 3]["date"].values[0]

waste_metrics_df = pd.DataFrame(columns = ['product', 'waste', 'lost_Revenue', 'lost_profit', 'cost_wasted'])

#for product in product_list:
filter_inventory = inventory[["day", "before or after delivery", "date", "week", product]]
promotions_filter = promotions[promotions["description"] == product]
filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)
#print(filter_inventory)
filtered_transactions = transactions[transactions["description"] == product]
def input_DOY(row):
    doy = pd.to_datetime(row["day"], dayfirst=True).dayofyear - 1
    return doy 
filtered_transactions["DOY"] = filtered_transactions.apply(input_DOY, axis=1)
cumsum_purchases = filtered_transactions.groupby("DOY").count()["product_id"].cumsum()
#creating a dataframe,  merging it with reference so each day is accounted for
df_purchases = pd.DataFrame({"DOY":cumsum_purchases.index, "cumulative purchases":cumsum_purchases})
ref = pd.DataFrame({"doy":range(1,365)}) #reference table to match up days
joint_purchases = ref.set_index('doy').join(df_purchases)
joint_purchases.fillna(method='pad', inplace=True)#days where 0 purchases happened

first_restock = df_date[df_date["day"] == 3]["date"].values[0]
#first_restock check purchases mde before the first restock
filtered_transactions[filtered_transactions["day"] < first_restock]

best_before = []
for element_list in filter_inventory[product]:
    for batch in element_list:
        best_before.append(batch[0])
bb_dates = np.unique(best_before)

df_waste = pd.DataFrame({"best before":bb_dates})
#df_waste #build initial df with points where waste can occur
df_waste["amount"] = df_waste.apply(input_inv, axis = 1)
#df_waste['rem_inventory'] = filter_inventory[(filter_inventory['before or after delivery'] == 'before') & (filter_inventory['day'].isin(df_waste['best before']))]['total inventory'] #- df_waste['amount']

df_waste['total_inventory'] = [int(filter_inventory.loc[filter_inventory[(filter_inventory['before or after delivery'] == 'before') & (filter_inventory['day'] == day)].index[0], 'total inventory']) if len(filter_inventory[(filter_inventory['before or after delivery'] == 'before') & (filter_inventory['day'] == day)].index) > 0 else None for day in df_waste['best before']]

df_waste['remaining_stock'] = df_waste['total_inventory'] - df_waste['amount']

print(filter_inventory[(filter_inventory['before or after delivery'] == 'before') & (filter_inventory['day'].isin(df_waste['best before']))])
df_waste = df_waste[df_waste["best before"] < 365] #only keep rows where expiry date is in same year

#first_restock check purchases mde before the first restock
filtered_transactions[filtered_transactions["day"] < first_restock]

#cumulative sum of purchases from that product
purchases_per_day = filtered_transactions.groupby("day").count()["product_id"].cumsum()
#purchases made per day from that product
purchases_per_day_plotting = filtered_transactions.groupby("day").count()["product_id"]
#idk know what is this
purchases_per_day_plotting.index = purchases_per_day_plotting.index.dayofyear/7

purchases_per_day.index[0].year
weeknr = []
for ind in purchases_per_day.index:
    week = datetime.date(ind.year, ind.month, ind.day).isocalendar()[1]
    weeknr.append(week)
#week number for each purchase day


df_waste.drop(index=df_waste.tail(9).index, inplace=True)
#drop the last rows not to cause index issues
#STILL NEED TO BE FIXED

df_waste["purchases"] = [joint_purchases['cumulative purchases'][i-1] for i in df_waste["best before"]]

df_waste["prev purchases"] = df_waste["purchases"].diff()

length = df_waste.shape[0]
lst_waste = [] #nr rows in waste dataframe
#amount left from the item
df_waste["remaining"] = df_waste["amount"] - df_waste["prev purchases"] 
#the first row is different as there is no prev pruchases available
df_waste["remaining"].iloc[0] = df_waste["amount"].iloc[0] - df_waste["purchases"].iloc[0]

for i in range(length):
    #iterates through the rows of the waste dataframe
    current_row = df_waste.iloc[i]
    if i == 0:
        #
        lst_waste.append(current_row["remaining"])
        continue     
    else:
        prev_row = df_waste.iloc[i-1] #previous row in dataframe
        if lst_waste[-1] < 0:
            #no idea whats happening here, I've had enough
            waste = lst_waste[-1] + current_row["remaining"]
            lst_waste.append(lst_waste[-1] + current_row["remaining"])
        else:
            lst_waste.append(current_row["remaining"])
#df_waste['total_inventory'] = 
df_waste["waste"] = lst_waste
def date_converter(row):
    converted_date = datetime.datetime(2018, 1, 1) + datetime.timedelta(row["best before"])
    return converted_date
purchase_price = products[products['description'] == product]['purchase_price']
df_waste["date"] = df_waste.apply(date_converter, axis = 1)
df_waste["week"] = df_waste.apply(week_of_year, axis = 1)
df_waste[df_waste["waste"] < 0]["waste"] = 0
df_waste['weekPrice'] = [float(products[products['description'] == product]['std_sales_price'])] * len(df_waste)
#df_waste.set_index('week', inplace = True)
#print(df_waste.head())
for week_ind in list(promotions[promotions['description'] == product].index):
    week = promotions.at[week_ind, 'week']
    #print(week_ind)
    # print(df_waste[['week']].dtypes)
    if week in list(np.unique(df_waste.week)):
        #print(week)
        df_waste.loc[df_waste[df_waste['week'] == week]['weekPrice'].index, 'weekPrice'] = (1 - (promotions.at[week_ind, 'discount'] / 100)) * df_waste.loc[df_waste[df_waste['week'] == week]['weekPrice'].index, 'weekPrice']
        #break
df_waste['lost_Revenue'] = df_waste.waste * df_waste.weekPrice
df_waste['purc_price'] = [float(purchase_price)] * len(df_waste)
df_waste['lost_profit'] = df_waste['waste'] * (df_waste['weekPrice'] - df_waste['purc_price'])
df_waste['cost_wasted'] = df_waste['waste'] * df_waste['purc_price']

<ipython-input-14-409aa1933388>:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_inventory["total inventory"] = filter_inventory.apply(total_inv, axis = 1)
<ipython-input-14-409aa1933388>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transactions["DOY"] = filtered_transactions.apply(input_DOY, axis=1)
     day before or after delivery       date  week  \
8     14                   before 2018-01-15     3   
10    17                   before 2018-01-18     3   
12    21                  

In [15]:
df_waste

,best before,amount,total_inventory,remaining_stock,purchases,prev purchases,remaining,waste,date,week,weekPrice,lost_Revenue,purc_price,lost_profit,cost_wasted
0,14,26,67.0,41.0,47.0,NaN,-21.0,-21.0,2018-01-15,3,1.79,-37.59,0.41,-28.98,-8.61
1,17,26,77.0,51.0,59.0,12.0,14.0,-7.0,2018-01-18,3,1.79,-12.53,0.41,-9.66,-2.87
2,21,26,86.0,60.0,77.0,18.0,8.0,1.0,2018-01-22,4,1.79,1.79,0.41,1.38,0.41
3,24,26,78.0,52.0,85.0,8.0,18.0,18.0,2018-01-25,4,1.79,32.22,0.41,24.84,7.38
4,28,26,85.0,59.0,104.0,19.0,7.0,7.0,2018-01-29,5,1.79,12.53,0.41,9.66,2.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,318,26,78.0,52.0,1159.0,8.0,18.0,18.0,2018-11-15,46,1.79,32.22,0.41,24.84,7.38
88,322,26,86.0,60.0,1177.0,18.0,8.0,8.0,2018-11-19,47,1.79,14.32,0.41,11.04,3.28
89,325,26,77.0,51.0,1182.0,5.0,21.0,21.0,2018-11-22,47,1.79,37.59,0.41,28.98,8.61
90,329,26,97.0,71.0,1189.0,7.0,19.0,19.0,2018-11-26,48,1.79,34.01,0.41,26.22,7.79


In [ ]:
#now make a column of inventory on the 14th (by taking inventory of last delivery day minus purchases since last delivery)
#count number of day less than the bb day to get index of last known inventory

In [ ]:
#find number of 